In [ ]:
import pandas as pd
import numpy as np
from datetime import date
import csv 
import requests
import glob
from pandas import DataFrame
from pandas import concat
from matplotlib import pyplot
from util import  select_relevant_rows, select_attributes, read_movement_data, read_multiple_csv

In [ ]:
path = r'/Users/marcomazzoli/Documents/Projects/COVID-19/dati-regioni/dpc-covid19-ita-regioni.csv'

frame = pd.read_csv(path)

In [ ]:
region_focus = 'Emilia-Romagna'
attribute_focus = 'denominazione_regione'

region_focus_data = select_relevant_rows(frame, attribute_focus, region_focus)

In [ ]:
list(region_focus_data)

In [ ]:
frame_interesting_columns = select_attributes(region_focus_data, ['data','ricoverati_con_sintomi','terapia_intensiva','totale_ospedalizzati','variazione_totale_positivi','nuovi_positivi','deceduti','tamponi','ingressi_terapia_intensiva'])

In [ ]:
frame_interesting_columns.tail()

In [ ]:
# 'tamponi' is cumulative, revert to incidence
column_to_normalize = 'tamponi'

frame_interesting_columns[column_to_normalize] = frame_interesting_columns[column_to_normalize].transform(lambda s: s.sub(s.shift().fillna(0)).abs())

In [ ]:
# 'tamponi' is cumulative, revert to incidence
column_to_normalize = 'deceduti'

frame_interesting_columns[column_to_normalize] = frame_interesting_columns[column_to_normalize].transform(lambda s: s.sub(s.shift().fillna(0)).abs())

In [ ]:
frame_interesting_columns = pd.DataFrame(frame_interesting_columns)
frame_interesting_columns['data'] = pd.to_datetime(frame_interesting_columns['data'])
frame_interesting_columns['data'] = frame_interesting_columns['data'].dt.strftime(r'%Y-%m-%d')


In [ ]:
path = '/Users/marcomazzoli/Documents/Scuola/Region_Mobility_Report_CSVs'
df = read_movement_data(path, 'IT_Region_Mobility_Report', region='Emilia-Romagna')

In [ ]:
"""
DataSet crea istanze della serie temporale, con variabili parametriche, ogni istanza ha i valori considerati (da file DataSet) per un valore parametrico di giorni antecedenti.
Provincia (regione) (si puo’ scendere per comune nel caso sia disponibile, ma se i valori sono inferiori a 30 difficile fare training).
Età in fasce (se disponibile, lavorare per fasce, numero per fascia considerata, almeno 4-5). (Assente da protezione civile.)
Genere (in due variabili, numero uomini, numero donne, se presente da dati IRST). (Assente da protezione civile.)
Numero di tamponi effettuati (tasso ogni 100.000 residenti).
Numero di tamponi effettuati su nuovi soggetti (tasso ogni 100.000 residenti).
Numero di tamponi positivi su nuovi soggetti (tasso ogni 100.000 residenti).
Numero di tamponi positivi su totale tamponi effettuati (tasso ogni 100.000 residenti).
Popolazione residente.
Densità di popolazione.
Dati mobilita’ di Google.

Output
Numero di pazienti ricoverati (tasso ogni 100.000 residenti).
Numero di pazienti ricoverati in terapia intensiva (tasso ogni 100.000 residenti).
Numero di decessi (tasso ogni 100.000 residenti).
"""

In [ ]:
list(df)

In [ ]:
df = df[['date',
            'retail_and_recreation_percent_change_from_baseline',
            'grocery_and_pharmacy_percent_change_from_baseline',
            'parks_percent_change_from_baseline',
            'transit_stations_percent_change_from_baseline',
            'workplaces_percent_change_from_baseline',
            'residential_percent_change_from_baseline']]

In [ ]:
df.head()

In [ ]:
frame_interesting_columns.rename(columns={'data': 'date'}, inplace=True)

In [ ]:
merged = pd.merge(frame_interesting_columns,df,how='outer',on='date')

In [ ]:
frame_interesting_columns.set_index('date',inplace=True)
df.set_index('date',inplace=True)

In [ ]:
frame_interesting_columns.head()

In [ ]:
merged = pd.merge(frame_interesting_columns,df,on='date')
merged = merged.fillna(0)

In [ ]:
list(merged)

In [ ]:
merged.head()

In [ ]:
dataset = merged
values = merged.values

dataset

groups = range(8)
i = 1
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups),1,i)
	pyplot.plot(values[:,group])
	pyplot.title(dataset.columns[group],y=0.5,loc='left')
	i += 1
pyplot.show()

In [ ]:
dataset = merged
values = merged.values

dataset

groups = range(8,14)
i = 1
pyplot.figure()
for group in groups:
	pyplot.subplot(len(groups),1,i)
	pyplot.plot(values[:,group])
	pyplot.title(dataset.columns[group],y=0.5,loc='left')
	i += 1
pyplot.show()

In [ ]:
# fix for package not found when installing in Anaconda environment
import pip
pip.main(['install', 'xgboost'])

In [ ]:
values = np.transpose(values)

In [ ]:
n_train_days = 20
train = values[:n_train_days, :]
test = values[n_train_days:, :]

train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

In [ ]:
from xgboost import XGBRegressor
model = XGBRegressor(objective='reg:squarederror', n_estimators=500)
model.fit(train_X, train_y)
yhat = model.predict(train_X)
yhat

In [ ]:
# evaluate an xgboost regression model on the housing dataset
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
import numpy as np
from xgboost import XGBRegressor

data = np.transpose(values)
n_output = 1

y_test = data[:, -1]
X_test = data[:, :-1]
# split data into input and output columns
X, y = data[:, :-n_output], data[:, -n_output]

In [ ]:
model = XGBRegressor()
model.fit(X, y)
# make a prediction
yhat = model.predict(X_test)
# summarize prediction
print('Predicted: ')
print(pd.DataFrame(yhat))
print('True: ')
print(pd.DataFrame(y_test))